In [39]:
import pandas as pd
import numpy as np
from pyemd import emd_samples
import plotly.graph_objects as go
from IPython.display import display

# To import packages from the parent directory
import sys
sys.path.insert(0,'..')

# Autoreload allows us to update our code mid-notebook
%load_ext autoreload
%autoreload 2

# Our code
import processing
import visualization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_pickle('../data/sample_morphologies.pickle')
print(f'Length of df: {len(df)}')

Length of df: 496


Collect final states

In [4]:
final_states = []
for grouping, group in df.groupby(['BR', 'CHI', 'version']):
    final_state = group[group.timestep == max(group.timestep)].iloc[0]
    final_states.append(final_state)
    
df_finals = pd.DataFrame(final_states)

Select 3 samples for analysis.

In [5]:
sample_x = df_finals.iloc[40]
sample_y = df_finals.iloc[8]
sample_z = df_finals.iloc[52]

sample_image_x = sample_x.image.reshape((100, 400))
sample_image_y = sample_y.image.reshape((100, 400))
sample_image_z = sample_z.image.reshape((100, 400))

fig_x = visualization.get_image_figure(sample_image_x)
fig_y = visualization.get_image_figure(sample_image_y)
fig_z = visualization.get_image_figure(sample_image_z)

print('sample_x')
print(sample_x)
display(fig_x)
print()

print('sample_y')
print(sample_y)
display(fig_y)
print()

print('sample_y')
print(sample_y)
display(fig_y)

sample_x
BR                                                       0.56
CHI                                                       3.4
count                                                     166
image       [0.05490177869796753, 0.471236914396286, 0.918...
timestep                                                  150
version                                                     0
Name: 224, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…


sample_y
BR                                                        0.5
CHI                                                       3.4
count                                                     287
image       [0.9672163724899292, 0.9605730772018433, 0.961...
timestep                                                  275
version                                                     0
Name: 394, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…


sample_y
BR                                                        0.5
CHI                                                       3.4
count                                                     287
image       [0.9672163724899292, 0.9605730772018433, 0.961...
timestep                                                  275
version                                                     0
Name: 394, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

Measure and store the signatures and measurements for the samples in the dataset.

In [16]:
df_sigs = df_finals.copy()
df_sigs['perimeter_area_ratio_sig'] = np.empty(len(df_sigs), dtype=np.object)
df_sigs['rect_area_ratio_sig'] = np.empty(len(df_sigs), dtype=np.object)
df_sigs['average_pixel_intensity_sig'] = np.empty(len(df_sigs), dtype=np.object)

df_sigs['average_pixel_intensity'] = np.empty(len(df_sigs), dtype=np.object)
df_sigs['fft'] = np.empty(len(df_sigs), dtype=np.object)

for idx, row in df_sigs.iterrows():
    comp_sample = row.image.reshape((100, 400))
    comp_components = processing.extract_components(comp_sample, background=1)    
    df_sigs.at[
        idx,
        'perimeter_area_ratio_sig'
    ] = processing.apply_to_components(comp_components, processing.perimeter_area_ratio_sig)
    
    df_sigs.at[
        idx,
        'rect_area_ratio_sig'
    ] = processing.apply_to_components(comp_components, processing.rect_area_ratio_sig)
    
    df_sigs.at[
        idx,
        'average_pixel_intensity_sig'
    ] = processing.apply_to_components(comp_components, processing.average_pixel_intensity_sig)
    
    df_sigs.at[
        idx,
        'average_pixel_intensity'
    ] = np.mean(comp_sample)
    
    df_sigs.at[
        idx,
        'fft'
    ] = np.fft.fft2(comp_sample)

# Similarity Query

Sort the dataset be most/least similar.

Set the query morphology.

In [78]:
sample_idx = 8
sample = df_sigs.iloc[sample_idx]
sample_image = sample.image.reshape((100, 400))

print(sample)
display(visualization.get_image_figure(sample_image))

BR                                                                           0.5
CHI                                                                          3.4
count                                                                        287
image                          [0.9672163724899292, 0.9605730772018433, 0.961...
timestep                                                                     275
version                                                                        0
perimeter_area_ratio_sig       [0.2814185431418234, 0.22127872410073543, 0.22...
rect_area_ratio_sig            [0.47039473684210525, 0.014538461538461538, 0....
average_pixel_intensity_sig    [0.7171052631578947, 0.3249230769230769, 0.576...
average_pixel_intensity                                                 0.499775
fft                            [[(19990.983684109524+0j), (9.378978828087622+...
Name: 394, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

Calculate distances between query sample signatures and database signatures.

In [79]:
dist_data = []

perimeter_area_ratio_sig = processing.apply_to_components(components, processing.perimeter_area_ratio_sig)
rect_area_ratio_sig = processing.apply_to_components(components, processing.rect_area_ratio_sig)

for idx, row in df_sigs.iterrows():
    dist_data.append({
        'BR': row.BR,
        'CHI': row.CHI,
        'version': row.version,
        'image': row.image,
        'emd_perimeter_area_ratio': emd_samples(row.perimeter_area_ratio_sig, sample.perimeter_area_ratio_sig),
        'emd_rect_area_ratio': emd_samples(row.rect_area_ratio_sig, sample.rect_area_ratio_sig),
        'emd_average_pixel_intensity': emd_samples(row.average_pixel_intensity_sig, sample.average_pixel_intensity_sig),
        'l2_average_pixel_intensity': np.linalg.norm(row.average_pixel_intensity - sample.average_pixel_intensity),
        'l2_fft': np.linalg.norm(row.fft - sample.fft)
    })

df_dist = pd.DataFrame(dist_data)

In [80]:
dist_list = [
    'emd_perimeter_area_ratio',
    'emd_rect_area_ratio',
    'emd_average_pixel_intensity',
    'l2_average_pixel_intensity',
    'l2_fft'
]

Display the most similar/dissimilar images for each signature.

In [81]:
for idx in range(4):
    print('-' * 30)
    print(f'Nearest: {idx}')
    print('-' * 30)
    print()
    for dist_name in dist_list:
        print(dist_name)        
        nearest = df_dist.sort_values(dist_name).iloc[idx]
        print(nearest)
        display(visualization.get_image_figure(nearest.image.reshape((100, 400))))
        print('\n\n====================================================\n\n')

------------------------------
Nearest: 0
------------------------------

emd_perimeter_area_ratio
BR                                                                           0.5
CHI                                                                          3.4
version                                                                        0
image                          [0.9672163724899292, 0.9605730772018433, 0.961...
emd_perimeter_area_ratio                                                       0
emd_rect_area_ratio                                                            0
emd_average_pixel_intensity                                                    0
l2_average_pixel_intensity                                                     0
l2_fft                                                                         0
Name: 8, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





emd_rect_area_ratio
BR                                                                           0.5
CHI                                                                          3.4
version                                                                        0
image                          [0.9672163724899292, 0.9605730772018433, 0.961...
emd_perimeter_area_ratio                                                       0
emd_rect_area_ratio                                                            0
emd_average_pixel_intensity                                                    0
l2_average_pixel_intensity                                                     0
l2_fft                                                                         0
Name: 8, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





emd_average_pixel_intensity
BR                                                                           0.5
CHI                                                                          3.4
version                                                                        0
image                          [0.9672163724899292, 0.9605730772018433, 0.961...
emd_perimeter_area_ratio                                                       0
emd_rect_area_ratio                                                            0
emd_average_pixel_intensity                                                    0
l2_average_pixel_intensity                                                     0
l2_fft                                                                         0
Name: 8, dtype: object


...

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





l2_average_pixel_intensity
BR                                                                           0.5
CHI                                                                          2.8
version                                                                        0
image                          [0.09411002695560455, 0.09400024265050888, 0.0...
emd_perimeter_area_ratio                                               0.0427765
emd_rect_area_ratio                                                    0.0988393
emd_average_pixel_intensity                                            0.0642731
l2_average_pixel_intensity                                           2.90456e-05
l2_fft                                                                   23165.2
Name: 4, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





l2_fft
BR                                                                           0.5
CHI                                                                          2.2
version                                                                        0
image                          [0.7294167280197144, 0.7301174998283386, 0.725...
emd_perimeter_area_ratio                                               0.0452845
emd_rect_area_ratio                                                    0.0608418
emd_average_pixel_intensity                                            0.0526264
l2_average_pixel_intensity                                           0.000249371
l2_fft                                                                   18464.4
Name: 0, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

In [82]:
for idx in range(3):
    print('-' * 30)
    print(f'Furthest: {idx}')
    print('-' * 30)
    print()
    for dist_name in dist_list:
        print(dist_name)        
        furthest = df_dist.sort_values(dist_name).iloc[-1-idx]
        print(furthest)
        display(visualization.get_image_figure(furthest.image.reshape((100, 400))))
        print('\n\n====================================================\n\n')

------------------------------
Furthest: 0
------------------------------

emd_perimeter_area_ratio
BR                                                                          0.56
CHI                                                                            4
version                                                                        0
image                          [0.002926476998254657, 0.023009920492768288, 0...
emd_perimeter_area_ratio                                                0.195283
emd_rect_area_ratio                                                    0.0357969
emd_average_pixel_intensity                                            0.0359693
l2_average_pixel_intensity                                             0.0603127
l2_fft                                                                   24515.7
Name: 44, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





emd_rect_area_ratio
BR                                                                          0.59
CHI                                                                          2.2
version                                                                        1
image                          [0.3056536018848419, 0.3736875057220459, 0.473...
emd_perimeter_area_ratio                                               0.0791789
emd_rect_area_ratio                                                     0.182706
emd_average_pixel_intensity                                             0.151424
l2_average_pixel_intensity                                             0.0901681
l2_fft                                                                   18374.9
Name: 49, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





emd_average_pixel_intensity
BR                                                                          0.59
CHI                                                                          2.2
version                                                                        2
image                          [0.6712865829467773, 0.6950507760047913, 0.710...
emd_perimeter_area_ratio                                               0.0826696
emd_rect_area_ratio                                                     0.175181
emd_average_pixel_intensity                                             0.155965
l2_average_pixel_intensity                                              0.090298
l2_fft                                                                   18655.1
Name: 50, dtype: object


...

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





l2_average_pixel_intensity
BR                                                                          0.59
CHI                                                                            4
version                                                                        3
image                          [0.9757997989654541, 0.9780877828598022, 0.957...
emd_perimeter_area_ratio                                                 0.14174
emd_rect_area_ratio                                                     0.139112
emd_average_pixel_intensity                                              0.11503
l2_average_pixel_intensity                                             0.0905926
l2_fft                                                                   24495.3
Name: 63, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…





l2_fft
BR                                                                          0.53
CHI                                                                            4
version                                                                        2
image                          [0.978229820728302, 0.9779106974601746, 0.9778...
emd_perimeter_area_ratio                                               0.0679454
emd_rect_area_ratio                                                    0.0459085
emd_average_pixel_intensity                                            0.0520612
l2_average_pixel_intensity                                             0.0303744
l2_fft                                                                   25001.7
Name: 30, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

## Distance Matrix

In [83]:
chi_values = df_sigs.CHI.unique()
br_values = df_sigs.BR.unique()
version_values = df_sigs.version.unique()

def get_dist_matrix(sigs_x, sigs_y=None):
    if sigs_y is None:
        sigs_y = sigs_x
    dist_matrix = []
    for y in sigs_y:
        row = []
        for x in sigs_x:
            row.append(emd_samples(x, y))
        dist_matrix.append(row)
    return dist_matrix

In [84]:
dist_name_pretty = {
    'emd_perimeter_area_ratio': 'Average EMD (Perimeter to Area Ratio)',
    'emd_rect_area_ratio': 'Average EMD (Rectangle Area Ratio)',
    'emd_average_pixel_intensity': 'Average EMD (Average Pixel Intensity)',
    'l2_average_pixel_intensity': 'Average L2 (Average Pixel Intensity)',
    'l2_fft': 'Average L2 (FFT)'
}

print(sample)
display(visualization.get_image_figure(sample_image))
print()

for dist_name in dist_list:
    print(dist_name)
    dist_matrix = []
    for chi in chi_values:
        row = []
        for br in br_values:
            version_dists = []
            for version in version_values:
                comp_sample = df_dist[
                    (df_dist.CHI == chi) &
                    (df_dist.BR == br) &
                    (df_dist.version == version)
                ].iloc[0]
                if (comp_sample[['CHI', 'BR', 'version']] == sample[['CHI', 'BR', 'version']]).all():
                    continue
                version_dists.append(comp_sample[dist_name])
            row.append(np.mean(version_dists))
        dist_matrix.append(row)
    
    title = f'''$\\text{{{dist_name_pretty[dist_name]}}}\\\\
    \\text{{Sample: }}(\phi={sample.BR}, \chi={sample.CHI}, \\text{{version}}={sample.version})$'''

    viz = visualization.get_distance_matrix_figure(
        dist_matrix,
        labels_x = br_values,
        labels_y = chi_values,
        scaleanchor=None,
        showticklabels=True,
        title=title,
        xtitle='$\phi$',
        ytitle='$\chi$'
    )
    display(viz)

BR                                                                           0.5
CHI                                                                          3.4
count                                                                        287
image                          [0.9672163724899292, 0.9605730772018433, 0.961...
timestep                                                                     275
version                                                                        0
perimeter_area_ratio_sig       [0.2814185431418234, 0.22127872410073543, 0.22...
rect_area_ratio_sig            [0.47039473684210525, 0.014538461538461538, 0....
average_pixel_intensity_sig    [0.7171052631578947, 0.3249230769230769, 0.576...
average_pixel_intensity                                                 0.499775
fft                            [[(19990.983684109524+0j), (9.378978828087622+...
Name: 394, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…


emd_perimeter_area_ratio


FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '106ee7b6-cc23-4771-9fbb-559fb3afe083',
 …

emd_rect_area_ratio


...

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '1fe743db-e8c2-47ff-a468-7e01e00b1ced',
 …

l2_average_pixel_intensity


FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'e52e1137-5300-4378-9f96-a70e9d8ea1c4',
 …

l2_fft


FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '61279d42-60a9-4f2e-aa1b-86f1a5f0781d',
 …